In [695]:
#Imports
import functions as func
import yaml


#Libraries
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error



%matplotlib inline

db_locations = func.import_yaml()

#load df
df = pd.read_csv(db_locations['data_clean']['train'])

In [697]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,Automatic,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,Automatic,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,Gasoline,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,Automatic,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Automatic,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Automatic,Black,Beige,None reported,Yes,97500


In [699]:
df.shape

(187765, 13)

In [701]:
# Drop model, id, and engine
df = df.drop(['id', 'model', 'engine', 'ext_col', 'int_col', 'clean_title', 'accident', 'transmission'], axis=1)

In [703]:
# List of categorical columns to apply dummy encoding
categorical_columns = ['brand', 'model_year', 'fuel_type']

# Apply pd.get_dummies()
df = pd.get_dummies(df, columns=categorical_columns)

# Convert the new boolean dummy columns to 1/0
df = df.astype({col: 'int' for col in df.select_dtypes(include='bool').columns})


df.head()

,milage,price,brand_Acura,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,brand_Bentley,brand_Bugatti,brand_Buick,...,model_year_2019,model_year_2020,model_year_2021,model_year_2022,model_year_2023,model_year_2024,fuel_type_Diesel,fuel_type_Electric,fuel_type_Gasoline,fuel_type_Hybrid
0,213000,4200,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,143250,4999,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,136731,13900,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,19500,45000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,7388,97500,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [705]:
df.shape

(187765, 97)

<br>
<br>

**Train Test Split**

In [707]:
features = df.drop(columns = ["price"]) # Features (everything except 'price')
target = df["price"]                    # Target variable ('price')

In [709]:
# perform the train-test split (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

In [711]:
# check the shape of the resulting sets
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (150212, 96)
y_train shape: (150212,)
X_test shape: (37553, 96)
y_test shape: (37553,)


<br>
<br>

**Standardization and Normalization**

In [713]:
# Create an instance of the normalizer
normalizer = MinMaxScaler()

In [715]:
normalizer.fit(X_train)

MinMaxScaler()

In [717]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [719]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_train_norm.head()

,milage,brand_Acura,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,brand_Bentley,brand_Bugatti,brand_Buick,brand_Cadillac,...,model_year_2019,model_year_2020,model_year_2021,model_year_2022,model_year_2023,model_year_2024,fuel_type_Diesel,fuel_type_Electric,fuel_type_Gasoline,fuel_type_Hybrid
0,0.304767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.188676,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.126451,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.217091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.056135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [721]:
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)
X_test_norm.head()

,milage,brand_Acura,brand_Alfa,brand_Aston,brand_Audi,brand_BMW,brand_Bentley,brand_Bugatti,brand_Buick,brand_Cadillac,...,model_year_2019,model_year_2020,model_year_2021,model_year_2022,model_year_2023,model_year_2024,fuel_type_Diesel,fuel_type_Electric,fuel_type_Gasoline,fuel_type_Hybrid
0,0.602371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.237837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.063966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.208447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.059126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Bagging and Pasting¶

In [723]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [725]:
bagging_reg.fit(X_train_norm, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

Evaluate model's performance

In [727]:
pred = bagging_reg.predict(X_test_norm)

print(f"MAE {mean_absolute_error(pred, y_test): .2f}")
print(f"RMSE {mean_squared_error(pred, y_test, squared=False): .2f}")
print(f"R2 score {bagging_reg.score(X_test_norm, y_test): .2f}")

MAE  20289.71
RMSE  67551.04


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


R2 score  0.14


### Random Forest

In [729]:
# Initialize a Random Forest
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)

In [731]:
# Training the model
forest.fit(X_train_norm, y_train)

RandomForestRegressor(max_depth=20)

In [733]:
# Evaluate the model

pred = forest.predict(X_test_norm)

print(f"MAE, {mean_absolute_error(pred, y_test): .2f}")
print(f"RMSE, {mean_squared_error(pred, y_test, squared=False): .2f}")
print(f"R2 score, {forest.score(X_test_norm, y_test): .2f}")

MAE,  21108.82
RMSE,  76278.68


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


R2 score, -0.10


### Adaptive Boosting

In [735]:
# Initialize an  Adaptive Boost model

ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

In [737]:
# Training the model
ada_reg.fit(X_train_norm, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                  n_estimators=100)

Evaluate the model

In [745]:
pred = ada_reg.predict(X_test_norm)

print(f"MAE, {mean_absolute_error(pred, y_test): .2f}")
print(f"RMSE, {mean_squared_error(pred, y_test, squared=False): .2f}")
print(f"R2 score, {ada_reg.score(X_test_norm, y_test): .2f}")

MAE,  23523.53
RMSE,  89966.33


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


R2 score, -0.53


### Decision Tree

In [747]:
# Initialize a Decision Tree instance
tree = DecisionTreeRegressor(max_depth=10)

In [751]:
# Training the model
tree.fit(X_train_norm, y_train)

DecisionTreeRegressor(max_depth=10)

In [759]:
# Evaluate the model

y_pred_test_dt = tree.predict(X_test_norm)

print(f"MAE, {mean_absolute_error(y_pred_test_dt, y_test): .2f}")
print(f"RMSE, {mean_squared_error(y_pred_test_dt, y_test, squared=False): .2f}")
print(f"R2 score, {tree.score(X_test_norm, y_test): .2f}")

MAE,  20955.56
RMSE,  73432.32
R2 score, -0.02


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
